In [1]:
#import utils
import pandas as pd
import spacy
#from utils import *
import time
import gspread

In [2]:
# example of what we want to write
#df_targets = pd.read_csv("/data/enrico_benedetti/nihongoexample/data/targets/target_words.csv")
df_targets = pd.read_csv("../../data/targets/target_words.csv")
row = df_targets.loc[0]
target_word = row['target_word']
context_sentence = row['context_sentence']
k=5
target_level = "N5"
sys_out_filename = f"../../evaluation/outputs/generation/llm_jp/{target_word}_{target_level}_.csv"
df = pd.read_csv(sys_out_filename)
df

sys_sentences = df['sentence'][1:k+1]
sys_sentences

1          彼は仕事で成功していないので、彼女から相手にされていない。
2        東と西が互いに相手を非難するプロパガンダ放送を流し合っていた。
3                 日本では、「相手」という言葉はよく使われる。
4                  あなたには何でも相談できる相手がいますね。
5    東と西はそれぞれ互いに相手を非難するプロパガンダ放送を流し合っていた。
Name: sentence, dtype: object

In [3]:
# https://docs.gspread.org/en/latest/oauth2.html#for-bots-using-service-account
gc = gspread.service_account()
#gc = gspread.service_account(filename='../../service_account_secret.json')

In [ ]:
sh = gc.open("Evaluation-sheet-jp-v3.0_gpt4")

In [4]:
#rows[13] # i have to cycle here and reconstruct the things. 

# maybe a function that processes one block is good:

def get_block_info(block_list: list):
    """Parses block read from google sheets"""
    target_levels = ['N1', 'N2', 'N3', 'N4', 'N5']
    k = 5
    block = block_list
    result = {}
    first_line = block[0]
    
    assert(first_line[0:4] == [' Context sentence','Target level','Target word','Block ID'])
    second_line = block[1]
    result['context_sentence'] = second_line[0]
    result['target_level'] = second_line[1]

    assert(result['target_level'] in target_levels)
    result['target_word'] = second_line[2]
    result['block_id'] = second_line[3]

    third_line = block[2]
    assert(len(third_line) >= 9)

    lists = block[4:4+k]
    sys_sentences = pd.DataFrame(lists, columns=['sentence','level_human','sense_human', 'reject_human']*3)

    sys_sentences = sys_sentences.rename(columns={0: 'sentence', 1: 'level_human', 2: 'sense_human', 3: 'reject_human'})
    # display(sys_sentences)
    result['sys_1'] = {}
    result['sys_2'] = {}
    result['sys_3'] = {}

    result['sys_1']['sentences'] = sys_sentences.iloc[:,0:3]
    result['sys_2']['sentences'] = sys_sentences.iloc[:,3:6]
    result['sys_3']['sentences'] = sys_sentences.iloc[:,6:9]
    
    diversity_line = [item for item in block[10] if item != '']
    result['sys_1']['diversity_human'] = diversity_line[0]
    result['sys_2']['diversity_human'] = diversity_line[1]
    result['sys_3']['diversity_human'] = diversity_line[2]

    rank_line = [item for item in block[12] if item != '']
    result['sys_1']['rank_human'] = rank_line[0]
    result['sys_2']['rank_human'] = rank_line[1]
    result['sys_3']['rank_human'] = rank_line[2]

    return result

In [5]:
# find how to analyze stuff... with this structure
# we should linearize it also
def get_block_info_flat(block_list: list):
    """Parses block read from google sheets into a one sentence per line dataframe."""
    target_levels = ['N1', 'N2', 'N3', 'N4', 'N5']
    k = 5
    block = block_list
    result = []
    first_line = block[0]
    second_line = block[1]
    assert(first_line[0:4] == [' Context sentence','Target level','Target word','Block ID']) # careful with space
    # need to build dataframe rows as 'target word / ctx / level / blockid / sys id / sentence / diff human /
    # sense human / reject / syn diversity (repeated) / ranking (repeated)' then we can group and count easily
    # later we need to add the annotator and other info

    for sys_id in [1,2,3]:
        for i in range(k):
            df_element = {}
            
            # this part is common to all
            df_element['target_word'] = second_line[2]
            df_element['context_sentence'] = second_line[0]
            df_element['target_level'] = second_line[1]
            df_element['block_id'] = second_line[3]
            df_element['sys_id'] = sys_id

            assert(df_element['target_level'] in target_levels)

            third_line = block[2]
            assert(len(third_line) >= 9)


            # this part is common to system but we replicate
            # reading normally fails when there is no annotation (last blank line does not get downloaded)
            # so we fill it automatically with blank
            df_element['diversity_human'] = block[10][0+sys_id-1+3*(sys_id-1)]
            try:
                df_element['rank_human'] = block[12][0+sys_id-1+3*(sys_id-1)]
            except IndexError:
                df_element['rank_human'] = ''

            # this part is unique - 5 should stay because everything is at the same height
            df_element['sentence'] = block[4+i][0+sys_id-1 + 3*(sys_id-1)]
            df_element['level_human'] = block[4+i][1+sys_id-1 + 3*(sys_id-1)]
            df_element['sense_human'] = block[4+i][2+sys_id-1 + 3*(sys_id-1)]
            df_element['reject'] = block[4+i][3+sys_id-1 + 3*(sys_id-1)]
            # fix cases where reject is blank -> transform into 'FALSE'
            if df_element['reject'] != 'TRUE' and df_element['rank_human'] != '':
                df_element['reject'] = 'FALSE'
            #otherwise it stays '' therefore nan later
            elif df_element['reject'] != 'TRUE': df_element['reject'] = ''

            if df_element['reject'] == 'TRUE':
                df_element['sense_human'] = 'rejected'
                df_element['level_human'] = 'rejected'
                
            result.append(df_element)
    return result

In [6]:
# function that gets all results from a file (and therefore all sheets)
# Rembemer to share with : human-eval@nii-thesis.iam.gserviceaccount.com
#sheet_name = 'Evaluation-sheet-jp-v3.0_gpt4' # it is gpt_4
#annotator_id = 'gpt4'
#sheet_name = 'Henri_Enrico_Evaluation-sheet-jp-v3.0' #  is human_learner_1
#annotator_id = 'human_learner_1'
#sheet_name = 'Willy_Enrico_Evaluation-sheet-jp-v3.0' #  is human_learner_2
#annotator_id = 'human_learner_2'
#sheet_name = 'Aizawa_Enrico_Evaluation-sheet-jp-v3.0' #  is human_native_1
#annotator_id = "human_native_1"
#sheet_name = 'Copia di takizawa - Enrico_Evaluation-sheet-jp-v3.0' #  is human_native_2
#annotator_id = 'human_native_2'
sheet_name = 'tsurusaki_Enrico_Evaluation-sheet-jp-v3.0' #  is human_native_3 tsurusaki_Enrico_Evaluation-sheet-jp-v3.0
annotator_id = 'human_native_3'

sh = gc.open(sheet_name)
# accumulate thing (here maybe add the annotator name)
sheet_result = []
for sheet_id in range(1,31):
    # get sheet
    eval_sheet = sh.worksheet(f"{sheet_id}")
    # get rows
    rows = eval_sheet.get_all_values()
    # use function
    sheet_result += get_block_info_flat(rows)

df_gpt4 = pd.DataFrame(sheet_result)

df_gpt4['ann_id'] = annotator_id
#df_gpt4.to_csv(f"../../evaluation/annotation/gpt/answers_{annotator_id}.csv", index=False)
df_gpt4.to_csv(f"../../evaluation/annotation/human/answers_{annotator_id}.csv", index=False)

In [ ]:
# in the case of gpt4, we want to look at the agreement between different runs (so the three blocks after the first)

sheet_name = 'Evaluation-sheet-jp-v3.0_gpt4'
sh = gc.open(sheet_name)
# accumulate thing (here maybe add the annotator name)
sheet_result = []
for sheet_id in range(1,31):
    # get sheet
    eval_sheet = sh.worksheet(f"{sheet_id}")
    # get rows
    rows = eval_sheet.get_all_values()
    time.sleep(20) # because of api request limit
    # use function to get the 3 annotations in each sheet (above the first)
    for i in range(3):
        offset = 20*(i+1)
        rows_spec_run = rows[0 + offset:13 + offset]
        df_elements = get_block_info_flat(rows_spec_run)

        # get a different id here
        annotator_id = f'gpt4_{i+1}'
        for el in df_elements:
            el['ann_id'] = annotator_id
        sheet_result += df_elements


df_gpt4 = pd.DataFrame(sheet_result)
#annotator_id = 'gpt4'
#df_gpt4['ann_id'] = annotator_id
df_gpt4.to_csv(f"../../evaluation/annotation/gpt/answers_gpt4_multiple.csv", index=False)

In [ ]:
df_gpt4.value_counts(subset=['rank_human'])